In [4]:
import csv 
import numpy
from scipy import integrate, linalg
from matplotlib import pyplot 
%matplotlib inline

In [18]:
#load geometry from data file 
with open('MainFoil_N=50.csv') as csvfile:
    naca_geom = csv.reader(csvfile, delimiter=',')